論文  
https://arxiv.org/abs/2202.04901  
  
GitHub  
https://github.com/google-research/frame-interpolation  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Frame_Interpolation_for_Large_Scene_Motion.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

In [ ]:
!nvidia-smi

# GitHubからFILMのソースコードを取得

In [ ]:
%cd /content/

!git clone https://github.com/google-research/frame-interpolation frame_interpolation


%cd /content/frame_interpolation
# Commits on Jan 14, 2023
!git checkout 69f8708f08e62c2edf46a27616a4bfcf083e2076

# ライブラリのインストール

In [ ]:
%cd /content/frame_interpolation

!pip install apache-beam==2.48.0 mediapy==1.1.8
!apt-get install ffmpeg
!pip install --upgrade gdown

# 学習済みモデルのロード

In [ ]:
%cd /content/frame_interpolation
!mkdir pretrained_models

!gdown https://drive.google.com/drive/folders/131_--QrieM4aQbbLWrUtbO2cGbX8-war -O ./pretrained_models --folder
!gdown https://drive.google.com/drive/folders/1gKgBfCQyOeweALLf7YAzWqh3ZkPgu5e5 -O ./pretrained_models --folder

# One mid-frame interpolation(1枚の中間フレーム補完)

In [ ]:
%cd /content/frame_interpolation

!python3 -m eval.interpolator_test \
     --frame1 ./photos/one.png \
     --frame2 ./photos/two.png \
     --model_path ./pretrained_models/Style/saved_model \
     --output_frame ./photos/middle.png

# Many in-between frames interpolation(複数枚の中間フレーム補完)

In [ ]:
%cd /content/frame_interpolation

!python3 -m eval.interpolator_cli \
     --pattern /content/frame_interpolation/photos \
     --model_path ./pretrained_models/Style/saved_model \
     --times_to_interpolate 6 \
     --output_video

In [ ]:
from moviepy.editor import *
from moviepy.video.fx.resize import resize

clip = VideoFileClip("./photos/interpolated.mp4")
# 表示用にリサイズ
clip = resize(clip, height=400)
clip.ipython_display()


# 任意の動画のフレームを切り出し中間フレームを補完させてみる
使用動画  
https://pixabay.com/ja/videos/%E5%A5%B3%E6%80%A7-%E7%AC%91%E9%A1%94-%E5%96%9C%E3%81%B3-%E5%B9%B8%E7%A6%8F-%E5%85%AC%E5%9C%92-83860/

In [ ]:
%cd /content/frame_interpolation/
!mkdir -p videos/frames
%cd videos

from google.colab import files

uploaded = files.upload()
uploaded = list(uploaded.keys())
file_name = uploaded[0]

## 動画をフレームに分割

In [ ]:
%cd /content/frame_interpolation/videos

!ffmpeg -i {file_name} /content/frame_interpolation/videos/frames/frame_%d.png

In [ ]:
# frame_1からframe_15の中間フレームを補完させる
!rm -rf /content/frame_interpolation/videos/target_frames
%cd /content/frame_interpolation/videos
!mkdir target_frames
!cp frames/frame_1.png target_frames
!cp frames/frame_15.png target_frames

In [ ]:
%cd /content/frame_interpolation

!python3 -m eval.interpolator_cli \
     --pattern "/content/frame_interpolation/videos/target_frames" \
     --model_path ./pretrained_models/Style/saved_model \
     --times_to_interpolate 6 \
     --output_video

In [ ]:
from moviepy.editor import *
from moviepy.video.fx.resize import resize

clip = VideoFileClip("/content/frame_interpolation/videos/target_frames/interpolated.mp4")
# 表示用にリサイズ
clip = resize(clip, height=520)
clip.ipython_display()
